# **PREPARAÇÃO**

Importar a lib do pandas:


In [1]:
import pandas as pd

Instalar libs, utilizar o `!pip install`:

In [ ]:
!pip install fuzzywuzzy

Importar a biblioteca fuzzywuzzy depois de instalado:

In [11]:
from fuzzywuzzy import fuzz

Ler os arquivos CSV, utilizar o comando `read_csv`

In [12]:
cadastro = pd.read_csv('https://raw.githubusercontent.com/renzoziegler/data_engineering_bootcamp/main/cadastro.csv')

In [13]:
dados_medicos = pd.read_csv('https://raw.githubusercontent.com/renzoziegler/data_engineering_bootcamp/main/dados_medicos.csv')

# **ANÁLISE EXPLORATORIA DOS DADOS**

Conhecendo os dados que serão analisadas e se estão no formato correto.

Exibir a quantidade de registros, utilizando o comando `print(frame.shape)`

In [ ]:
print(cadastro.shape)
print(dados_medicos.shape)

Exibir as colunas do frame informado, utilizando o comando `print(frame.columns)`:

In [ ]:
print(cadastro.columns)
print(dados_medicos.columns)

O comando `head` visualiza as primeiras linhas:

In [ ]:
cadastro.head()

O comando `iloc[num]` exibi a linha informada:

In [ ]:
print(cadastro.iloc[0])

O comando `dtypes` exibe os datatypes das colunas do frame

In [ ]:
print(cadastro.dtypes)

# **PRÉ PROCESSAMENTO**

Tratando o campo CPF

- `astype(formato)` para transformar o tipo do dados para o indicado entre parenteses.

- `zfill(qtd)` para preencher a coluna com a quantidade faltante, conforme o valor informado (qtd).



In [ ]:
dados_medicos.cpf.astype(str).str.zfill(11)

Tratar o campo CPF utilizando a função `lambda`.

`lambda` é uma função anonima no phyton, é usada para definir em tempo de execução.

In [30]:
cadastro.cpf = cadastro.cpf.astype(str).apply(lambda x: x.zfill(11))
dados_medicos.cpf = dados_medicos.cpf.astype(str).apply(lambda x: x.zfill(11))

In [ ]:
print(cadastro.cpf)
print(dados_medicos.cpf)

Quebrar NOME em nome e sobrenome, utilizando o comando `split(separador)`



In [26]:
cadastro['nome_array'] = cadastro.nome.str.split()
cadastro['primeiro_nome'] = cadastro.nome_array.str[0]
cadastro['sobrenome'] = cadastro.nome_array.str[-1]

dados_medicos['nome_array'] = dados_medicos.nome.str.split()
dados_medicos['primeiro_nome'] = dados_medicos.nome_array.str[0]
dados_medicos['sobrenome'] = dados_medicos.nome_array.str[-1]

In [ ]:
print(cadastro.primeiro_nome)

In [ ]:
print(cadastro.sobrenome)

Converter para datatime DATA DE NASCIMENTO 

In [ ]:
cadastro.data_nasc = pd.to_datetime(cadastro.data_nasc, format = '%Y-%m-%d')
dados_medicos.data_nasc = pd.to_datetime(dados_medicos.data_nasc, format = '%Y/%m/%d')

print(cadastro.data_nasc)
print(dados_medicos.data_nasc)

Podemos quebrar em ano, mes e dia para poder comparar em outra análise

In [ ]:
cadastro['nasc_ano'] = cadastro.data_nasc.apply(lambda x : x.year)
cadastro['nasc_mes'] = cadastro.data_nasc.apply(lambda x : x.month)
cadastro['nasc_dia'] = cadastro.data_nasc.apply(lambda x : x.day)

dados_medicos['nasc_ano'] = dados_medicos.data_nasc.apply(lambda x : x.year)
dados_medicos['nasc_mes'] = dados_medicos.data_nasc.apply(lambda x : x.month)
dados_medicos['nasc_dia'] = dados_medicos.data_nasc.apply(lambda x : x.day)

print(cadastro.data_nasc.iloc[0])
print('Cadastro Ano: ', cadastro.nasc_ano.iloc[0])
print('Cadastro Mes: ', cadastro.nasc_mes.iloc[0])
print('Cadastro Dia: ', cadastro.nasc_dia.iloc[0])

print(dados_medicos.data_nasc.iloc[0])
print('dados_medicos Ano: ', dados_medicos.nasc_ano.iloc[0])
print('dados_medicos Mes: ', dados_medicos.nasc_mes.iloc[0])
print('dados_medicos Dia: ', dados_medicos.nasc_dia.iloc[0])

Converter altura em número.

`str.replace(old, new[, max])`

> old -- substring a ser modificada

> new -- a nova substring

> max -- número de vezes que será substituída







In [ ]:
dados_medicos.altura = dados_medicos.altura.str.replace(',','.').astype(float)

print(dados_medicos.altura)

# **INDEXAÇÃO**

Passo importante para ajudar a diminuir o número de comparações e evitar comparações desnecessárias. Ganho computacional.

Lista possíveis blocking key (sorting key) e definir uma a ser utilizada.

Criar os blocos os registros em bloco onde possíveis matches estão presentes.


In [ ]:
cadastro.groupby('nasc_mes').size()
dados_medicos.groupby('nasc_mes').size() #Muitas pessoas com nascimento em 1900
dados_medicos[dados_medicos['nasc_mes'] == 1]['data_nasc'].sort_values(ascending = False)

blocos = pd.concat([cadastro.groupby('nasc_mes').size(), dados_medicos.groupby('nasc_mes').size()], axis = 1)
blocos.columns = ['cadastro', 'dados_medicos']
print(sum(blocos.cadastro * blocos.dados_medicos)) #12.254 ao invés de 144.624
blocos.head()

In [ ]:
dados_medicos[dados_medicos['data_nasc'] == '1900-01-01']

In [ ]:
cadastro['cpf_0'] = cadastro.cpf.str[0]
cadastro.groupby('cpf_0').size()

In [ ]:
dados_medicos['cpf_0'] = dados_medicos.cpf.str[0]
dados_medicos.groupby('cpf_0').size()

In [ ]:
cadastro[cadastro['cpf_0'] == 0]

In [ ]:
blocos_cpf = pd.concat([cadastro.groupby('cpf_0').size(), dados_medicos.groupby('cpf_0').size()], axis = 1)
blocos_cpf.columns = ['cadastro', 'dados_medicos']
print(sum(blocos_cpf.cadastro * blocos_cpf.dados_medicos)) #14.862 ao invés de 144.624

In [ ]:
cadastro_blocos

In [88]:
dados_medicos_cpf_limpo = dados_medicos[dados_medicos['cpf'] != '00000000000']

In [89]:
#criando disiconários vazios (no dicionario tenho 1 indice para cada registro):
cadastro_blocos = {}
dados_medicos_blocos = {}

#dividir em listas:
for i in range(10):
    cadastro_blocos[i] = cadastro[cadastro['cpf_0'].astype(int) == i]
    dados_medicos_blocos[i] = dados_medicos_cpf_limpo[dados_medicos_cpf_limpo['cpf_0'].astype(int) == i]

In [ ]:
dados_medicos_blocos

# **COMPARAÇÃO**

Nessa sessão iremos comparar cada grupo, para verificar se são iguais ou não

Utilização da biblioteca fuzzyWuzzy - função fuzzy.token_sort_ratio : primeiro é ordenado e em seguida cada token é comparado.

In [103]:
#Comparação par a par
def comparaRegistros(registro1, registro2):
    #cria dicionário com similaridades
    sim = {}
    #Compara o CPF, que são exatamente igual
    if registro1['cpf'] == registro2['cpf']:
        sim['cpf'] = 1
    else:
        sim['cpf'] = 0
    
    #Compara Data de Nascimento, que são exatamente igual
    if registro1['data_nasc'] == registro2['data_nasc']:
        sim['data_nasc'] = 1
    elif (registro1['nasc_ano'] == registro2['nasc_ano']) and (registro1['nasc_mes'] == registro2['nasc_dia']) and (registro1['nasc_dia'] == registro2['nasc_mes']):
        #Mês e dia trocado!!
        sim['data_nasc'] = 1
    else:
        sim['data_nasc'] = 0

    #Compara Nomes, que possuem similaridade de strings:
    if registro1['nome'] == registro2['nome']:
        sim['nome'] = 1
    else:
        sim['nome'] = fuzz.token_sort_ratio(registro1['nome'], registro2['nome'])/100
        
    return sim

In [105]:
pares = pd.DataFrame(columns = ['paciente', 'registro', 'similaridade'])

for i in range(0,10):
    print('CPF começando com {}'.format(i))
    #Varrer todos os dados_medicos, comparando com os registros em cadastro:
    for k, paciente in dados_medicos_blocos[i].iterrows():
        for j, registro in cadastro_blocos[i].iterrows():
            #print('Dados médicos: {}, {}, {}'.format(paciente.nome, paciente.cpf, paciente.data_nasc))
            #print('Cadastro: {}, {}, {}'.format(registro.nome, registro.cpf, registro.data_nasc))
            sim = comparaRegistros(paciente, registro)
            #print('Similaridade: {}, {}, {}'.format(sim['nome'], sim['cpf'], sim['data_nasc']))
            par = {'paciente' : paciente, 'registro' : registro, 'similaridade' : sim}
            pares = pares.append(par, ignore_index = True)

pares['simSum'] = 0

matches = pd.DataFrame()
matches_registros = pd.DataFrame()
potenciais = pd.DataFrame()
potenciais_registros = pd.DataFrame()
non_matches = pd.DataFrame()
non_matches_registros = pd.DataFrame()

CPF começando com 0
CPF começando com 1
CPF começando com 2
CPF começando com 3
CPF começando com 4
CPF começando com 5
CPF começando com 6
CPF começando com 7
CPF começando com 8
CPF começando com 9


In [106]:
pares.head()

,paciente,registro,similaridade,simSum
0,nome Lucca Carlos Ferreira da...,nome Lúcia Carolina Luz...,"{'cpf': 0, 'data_nasc': 0, 'nome': 0.47}",0
1,nome Lucca Carlos Ferreira da...,nome Sabrina Apare...,"{'cpf': 0, 'data_nasc': 0, 'nome': 0.37}",0
2,nome Lucca Carlos Ferreira da...,nome Igor ...,"{'cpf': 0, 'data_nasc': 0, 'nome': 0.35}",0
3,nome Lucca Carlos Ferreira da...,nome Tia...,"{'cpf': 0, 'data_nasc': 0, 'nome': 0.32}",0
4,nome Lucca Carlos Ferreira da...,nome Gustavo Ant...,"{'cpf': 0, 'data_nasc': 0, 'nome': 0.3}",0


# **CLASSIFICAÇÃO**

In [145]:
#Classificação
for idx, par in pares.iterrows():
    sim = par['similaridade']
    simSum = sim['cpf'] + sim['nome'] + sim['data_nasc']
    #Possibilidades - dar peso diferentes aos critérios
    pares.loc[idx, 'simSum'] = simSum
    if simSum == 2:
        matches = matches.append(pares.loc[idx])
        matches_registros = matches_registros.append(pares.loc[idx, 'registro'])
    elif simSum >= 1.5:
        potenciais = potenciais.append(pares.loc[idx])
        potenciais_registros = potenciais_registros.append(pares.loc[idx, 'registro'])
    else:
        non_matches = non_matches.append(pares.loc[idx])
        non_matches_registros = non_matches_registros.append(pares.loc[idx, 'registro'])

is_missing = dados_medicos.merge(matches_registros, how = 'left', 
                on = ['nome', 'cpf', 'data_nasc'], indicator = True)
is_missing = is_missing[is_missing['_merge'] =='left_only'][['nome', 'data_nasc', 'cpf']]

In [161]:
print('matches: ', matches.shape)
print('potenciais: ', potenciais.shape)
print('non_matches: ', non_matches.shape)

matches:  (315, 4)
potenciais:  (1467, 4)
non_matches:  (72144, 4)


In [ ]:
#casos que precisamos rever manualmente:
potenciais.head(10)

In [ ]:
potenciais[potenciais['simSum'] == 2].iloc[0].paciente

In [ ]:
potenciais[potenciais['simSum'] == 2].iloc[0].registro

# **AVALIAÇÃO**

In [159]:
#Avaliação
%load_ext google.colab.data_table
potenciais[potenciais['simSum'] >= 1.8].sample(10)
#True Positive: 10
#False Positive: 0

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


,paciente,registro,simSum,similaridade
10236,nome Valentina Stefany Alana...,nome Valentina S...,2.98,"{'cpf': 1, 'data_nasc': 1, 'nome': 0.98}"
6167,nome Henrique Marcos Peixoto ...,nome Henrique Ma...,3.00,"{'cpf': 1, 'data_nasc': 1, 'nome': 1}"
9983,nome Enrico Iago Barbosa data...,nome ...,3.00,"{'cpf': 1, 'data_nasc': 1, 'nome': 1}"
3007,nome Giovanna Milena Costa da...,nome Giovanna...,3.00,"{'cpf': 1, 'data_nasc': 1, 'nome': 1}"
11876,nome Ruan Theo Duarte data_na...,nome Ruan Theo Duarte dat...,2.00,"{'cpf': 1, 'data_nasc': 0, 'nome': 1}"
1152,nome Anthoni Raimundo Corte ...,nome Anthony Rai...,1.96,"{'cpf': 1, 'data_nasc': 0, 'nome': 0.96}"
10236,nome Valentina Stefany Alana...,nome Valentina S...,2.98,"{'cpf': 1, 'data_nasc': 1, 'nome': 0.98}"
4285,nome César Edson Assuncão dat...,nome ...,2.97,"{'cpf': 1, 'data_nasc': 1, 'nome': 0.97}"
5177,nome Helena Isabella Fabiana...,nome Helena Isabe...,2.97,"{'cpf': 1, 'data_nasc': 1, 'nome': 0.97}"
4732,nome Alexandre Gustavo Leona...,nome Alexandre Gustavo ...,2.92,"{'cpf': 1, 'data_nasc': 1, 'nome': 0.92}"


In [157]:
non_matches[non_matches['simSum']> 1.3].sample(10)
#True Negative: 2
#False Negative: 8

,paciente,registro,simSum,similaridade
10490,nome Davi Francisco Isaac Ma...,nome Davi Francisco Is...,1.94,"{'cpf': 1, 'data_nasc': 0, 'nome': 0.94}"
10378,nome Maite Luna Ayla Nascime...,nome Maitê Luna Ayla Nascime...,1.98,"{'cpf': 1, 'data_nasc': 0, 'nome': 0.98}"
807,nome Aparecida Mariah Cunha d...,nome Apare...,1.94,"{'cpf': 1, 'data_nasc': 0, 'nome': 0.94}"
259,nome Lúcia C Luzia Almada da...,nome Lúcia Carolina Luz...,1.84,"{'cpf': 1, 'data_nasc': 0, 'nome': 0.84}"
259,nome Lúcia C Luzia Almada da...,nome Lúcia Carolina Luz...,1.84,"{'cpf': 1, 'data_nasc': 0, 'nome': 0.84}"
892,nome Gustavo Anthony Santos d...,nome Gustavo Ant...,1.92,"{'cpf': 1, 'data_nasc': 0, 'nome': 0.92}"
892,nome Gustavo Anthony Santos d...,nome Gustavo Ant...,1.92,"{'cpf': 1, 'data_nasc': 0, 'nome': 0.92}"
6925,nome Enzo Albuquerque Silva d...,nome Antonio Albuquerque Silva...,1.85,"{'cpf': 1, 'data_nasc': 0, 'nome': 0.85}"
8881,nome Camila Giovana Natália ...,nome Camila Giova...,1.97,"{'cpf': 1, 'data_nasc': 0, 'nome': 0.97}"
8625,nome Sueli Clara Hadassa Cru...,nome Sueli Clara H...,1.94,"{'cpf': 1, 'data_nasc': 0, 'nome': 0.94}"


In [160]:
#True Positive: 10
#False Positive: 0
#True Negative: 2
#False Negative: 8
TP = 10
FP = 0
TN = 2
FN = 8

acuracia = (TP+TN)/(TP+FP+TN+FN)
precisao = TP/(TP+FP)
recall = TP/(TP+FN)
fmeasure = 2*(precisao*recall)/(precisao+recall)

print("Acuracia: {}".format(acuracia))
print("Recall: {}".format(recall))
print("Precisão: {}".format(precisao))
print("F-measure: {}".format(fmeasure))

Acuracia: 0.6
Recall: 0.5555555555555556
Precisão: 1.0
F-measure: 0.7142857142857143
